In [5]:
import numpy as np
import pandas as pd 
import re
import nltk
import nltk.corpus
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import pickle

dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")
example = dataset['train'][0]
df = pd.DataFrame(dataset['train'])

df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(nltk.corpus.stopwords.words("english")) - set(["not"])

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
df['Tweet'] = df['Tweet'].apply(clean_text)
df['Tweet'] = df['Tweet'].str.replace('\d+', '')

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Tweet'].values)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(df['Tweet'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

df.iloc[:, 2:] = df.iloc[:, 2:].astype(int)
temp_column = []
for row in range(len(df)):
  temp_list = []
  for emo in range(len(df.iloc[row])):
    if(df.iloc[row][emo] == True):
      temp_list.append(1)
    if(df.iloc[row][emo] == False):
       temp_list.append(0)
  temp_list.insert(0, 0)
  temp_list.insert(0, 0)
  temp_column.append(temp_list)
df['Sentiments'] = temp_column

X = tokenizer.texts_to_sequences(df['Tweet'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
temp_column = pd.DataFrame(temp_column)
Y = temp_column.values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
y_train = np.asarray(Y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(Y_test).astype('float32').reshape((-1,1))
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

import tensorflow as tf
from tensorflow import keras
from keras.layers import (Dense, LSTM, Embedding, SpatialDropout1D)
from keras.models import Sequential
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

filepath = 'nnshanti.sav'
pickle.dump(model, open(filepath, 'wb'))

# new_complaint = ['I am really excited and love my girlfriend.']
# seq = tokenizer.texts_to_sequences(new_complaint)
# padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
# pred = model.predict(padded)
# labels = ['bruh', 'bruh', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
# print(pred, labels[np.argmax(pred)])

[nltk_data] Downloading package punkt to /home/x-akolli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/x-akolli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/x-akolli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_4191071/4156330684.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(df.iloc[row][emo] == True):
/tmp/ipykernel_4191071/4156330684.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(df.iloc[row][emo] == False):

(6154, 250) (6154, 13)
(684, 250) (684, 13)
Epoch 1/10
87/87 [==============================] - 11s 101ms/step - loss: 0.4385 - accuracy: 0.2111 - val_loss: 0.3904 - val_accuracy: 0.0844
Epoch 2/10
87/87 [==============================] - 8s 95ms/step - loss: 0.3925 - accuracy: 0.2555 - val_loss: 0.3672 - val_accuracy: 0.2760
Epoch 3/10
87/87 [==============================] - 8s 97ms/step - loss: 0.3333 - accuracy: 0.3852 - val_loss: 0.3237 - val_accuracy: 0.4205
Epoch 4/10
87/87 [==============================] - 9s 98ms/step - loss: 0.2879 - accuracy: 0.4745 - val_loss: 0.3219 - val_accuracy: 0.4399
Epoch 5/10
87/87 [==============================] - 8s 97ms/step - loss: 0.2585 - accuracy: 0.5560 - val_loss: 0.3099 - val_accuracy: 0.4821
Epoch 6/10
87/87 [==============================] - 9s 99ms/step - loss: 0.2313 - accuracy: 0.5807 - val_loss: 0.3144 - val_accuracy: 0.5081
Epoch 7/10
87/87 [==============================] - 8s 95ms/step - loss: 0.2116 - accuracy: 0.6038 - val_los